In [80]:
%matplotlib inline
%pwd

'/home/gabe/work/fast-ai/nbs'

In [81]:
%%html
<style>
  .end_space {
      min-height: 1000px;
  }
  .container {
      width: 100%;
  }
</style

In [82]:
import sys
import os
from importlib import reload

sys.path.append('../src')

In [83]:
DATA_DIR = '../data/statefarm'
SAMPLE_DIR = os.path.join(DATA_DIR, 'sample')
BATCH_SIZE = 64

In [84]:
import utils.statefarm
reload(utils.statefarm)

import utils.trainhelper
reload(utils.trainhelper)

from utils.statefarm import *
from utils.trainhelper import get_batches, save_model, read_model, get_classes
from utils.utils import save_array, load_array

from models.vgg16 import VGG16

In [85]:
from keras.models import Model
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import BatchNormalization
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.preprocessing import image

### Read train data

In [86]:
data_dir = DATA_DIR

width_igen = image.ImageDataGenerator(width_shift_range=0.1)
height_igen = image.ImageDataGenerator(height_shift_range=0.05)
shear_igen = image.ImageDataGenerator(shear_range=0.1)
rotation_igen = image.ImageDataGenerator(rotation_range=15)
channel_igen = image.ImageDataGenerator(channel_shift_range=20)
igen = image.ImageDataGenerator(rotation_range=15, 
                                height_shift_range=0.05, 
                                shear_range=0.1, 
                                channel_shift_range=20, 
                                width_shift_range=0.1)

train_batches = get_batches(os.path.join(data_dir, 'train'), batch_size=BATCH_SIZE, shuffle=False)
valid_batches = get_batches(os.path.join(data_dir, 'valid'), batch_size=BATCH_SIZE * 2, shuffle=False)
test_batches = get_batches(os.path.join(data_dir, 'test'), batch_size=BATCH_SIZE, shuffle=False, class_mode=None)

train_steps = int(np.ceil(train_batches.samples / BATCH_SIZE))
valid_steps = int(np.ceil(valid_batches.samples / (BATCH_SIZE * 2)))
test_steps = int(np.ceil(test_batches.samples / BATCH_SIZE))

print(train_batches.filenames[:5], 'filenames')
print(train_batches.num_class, 'classes')
print(train_batches.image_shape, 'image shape')

Found 21601 images belonging to 10 classes.
Found 823 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.
['c0/img_82409.jpg', 'c0/img_95245.jpg', 'c0/img_88538.jpg', 'c0/img_14492.jpg', 'c0/img_81194.jpg'] filenames
10 classes
(224, 224, 3) image shape


### Use the convolutional layers of VGG16 to generate outputs to the next model

In [87]:
vgg = VGG16(include_top=False, pooling=None, input_shape=(224,224,3))

train_vgg_preds = vgg.predict_generator(train_batches, train_steps)
valid_vgg_preds = vgg.predict_generator(valid_batches, valid_steps)
test_vgg_preds = vgg.predict_generator(test_batches, test_steps)


In [70]:
print(train_vgg_preds.shape, 'train_vgg_preds.shape')
print(valid_vgg_preds.shape, 'valid_vgg_preds.shape')
print(test_vgg_preds.shape, 'test_vgg_preds.shape')
print(vgg.layers[-1].output_shape[1:])

(21601, 7, 7, 512) train_vgg_preds.shape
(823, 7, 7, 512) valid_vgg_preds.shape
(79726, 7, 7, 512) test_vgg_preds.shape
(7, 7, 512)


In [88]:
CACHE_DIR = os.path.join(data_dir, 'cache')

save_array(os.path.join(CACHE_DIR, 'train_vgg_preds.dat'), train_vgg_preds)
save_array(os.path.join(CACHE_DIR, 'valid_vgg_preds.dat'), valid_vgg_preds)
save_array(os.path.join(CACHE_DIR, 'test_vgg_preds.dat'), test_vgg_preds)

In [89]:
(
   train_classes,
   valid_classes,
   train_labels,
   valid_labels,
   train_filenames,
   valid_filenames,
   test_filenames
) = get_classes(data_dir)


Found 21601 images belonging to 10 classes.
Found 823 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


In [66]:
print(len(train_classes))
print(len(valid_classes))
print(len(train_labels))
print(len(valid_labels))

21601
823
21601
823


### Setup model and train

In [108]:
DROPOUT_RATE = 0.9

model = Sequential([
    Flatten(input_shape=vgg.layers[-1].output_shape[1:]),
    Dropout(DROPOUT_RATE),
    
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(DROPOUT_RATE),
    
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(DROPOUT_RATE),
    
    Dense(10, activation='softmax')
])

model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_vgg_preds, train_labels, batch_size=BATCH_SIZE, epochs=10, validation_data=(valid_vgg_preds, valid_labels))

model.optimizer.lr = 0.001
model.fit(train_vgg_preds, train_labels, batch_size=BATCH_SIZE, epochs=10, validation_data=(valid_vgg_preds, valid_labels))

Train on 21601 samples, validate on 823 samples
Epoch 1/10
21601/21601 [==============================] - 7s - loss: 6.5389 - acc: 0.0998 - val_loss: 2.2983 - val_acc: 0.0595
Epoch 2/10
21601/21601 [==============================] - 6s - loss: 5.8983 - acc: 0.1092 - val_loss: 2.1577 - val_acc: 0.2139
Epoch 3/10
21601/21601 [==============================] - 6s - loss: 5.3204 - acc: 0.1226 - val_loss: 2.0730 - val_acc: 0.2734
Epoch 4/10
21601/21601 [==============================] - 6s - loss: 4.8276 - acc: 0.1268 - val_loss: 1.9901 - val_acc: 0.3609
Epoch 5/10
21601/21601 [==============================] - 6s - loss: 4.3340 - acc: 0.1401 - val_loss: 1.9271 - val_acc: 0.4083
Epoch 6/10
21601/21601 [==============================] - 6s - loss: 3.9083 - acc: 0.1502 - val_loss: 1.8739 - val_acc: 0.4070
Epoch 7/10
21601/21601 [==============================] - 6s - loss: 3.5191 - acc: 0.1634 - val_loss: 1.8190 - val_acc: 0.3998
Epoch 8/10
21601/21601 [==============================] - 6s - 

In [109]:
model.optimizer.lr = 1e-3
model.fit(train_vgg_preds, train_labels, batch_size=BATCH_SIZE, epochs=10, validation_data=(valid_vgg_preds, valid_labels))

Train on 21601 samples, validate on 823 samples
Epoch 1/10
21601/21601 [==============================] - 6s - loss: 1.2385 - acc: 0.5534 - val_loss: 1.1154 - val_acc: 0.6768
Epoch 2/10
21601/21601 [==============================] - 6s - loss: 1.1772 - acc: 0.5748 - val_loss: 1.0355 - val_acc: 0.7375
Epoch 3/10
21601/21601 [==============================] - 6s - loss: 1.1049 - acc: 0.6008 - val_loss: 0.9859 - val_acc: 0.7643
Epoch 4/10
21601/21601 [==============================] - 6s - loss: 1.0465 - acc: 0.6250 - val_loss: 0.9621 - val_acc: 0.7558
Epoch 5/10
21601/21601 [==============================] - 6s - loss: 0.9901 - acc: 0.6463 - val_loss: 0.9137 - val_acc: 0.7643
Epoch 6/10
21601/21601 [==============================] - 6s - loss: 0.9423 - acc: 0.6632 - val_loss: 0.9080 - val_acc: 0.7546
Epoch 7/10
21601/21601 [==============================] - 6s - loss: 0.8899 - acc: 0.6917 - val_loss: 0.8768 - val_acc: 0.7533
Epoch 8/10
21601/21601 [==============================] - 6s - 

In [57]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_7 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 25088)             0         
_________________________________________________________________
dense_17 (Dense)             (None, 256)               6422784   
_________________________________________________________________
batch_normalization_12 (Batc (None, 256)               1024      
_________________________________________________________________
dropout_18 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 256)               65792     
_________________________________________________________________
batch_normalization_13 (Batc (None, 256)               1024      
__________

### Save model

In [110]:
save_model(data_dir, model)

### Test model

In [112]:
test_preds = model.predict(test_vgg_preds, batch_size=BATCH_SIZE)

In [111]:
test_dir = os.path.join(data_dir, 'test')
test_batches = get_batches(test_dir, shuffle=False, batch_size=BATCH_SIZE, class_mode=None)
test_steps = int(np.ceil(test_batches.samples / BATCH_SIZE))
test_preds = model.predict_generator(test_batches, test_steps)

Found 79726 images belonging to 1 classes.


ValueError: Error when checking : expected flatten_22_input to have shape (None, 7, 7, 512) but got array with shape (64, 224, 224, 3)

In [113]:
valid_preds = model.predict(valid_vgg_preds, batch_size=BATCH_SIZE*2)

In [169]:
valid_dir = os.path.join(data_dir, 'valid')
valid_batches = get_batches(valid_dir, batch_size=BATCH_SIZE * 2, shuffle=False)
valid_steps = int(np.ceil(valid_batches.samples / (BATCH_SIZE * 2)))
valid_preds = model.predict_generator(valid_batches, valid_steps)

Found 823 images belonging to 10 classes.


### Save predictions

In [114]:
results_dir = os.path.join(data_dir, 'results')
save_array(os.path.join(results_dir, 'predictions'), test_preds)
save_array(os.path.join(results_dir, 'filenames'), test_batches.filenames)

### Prepare data for submission

In [115]:
from keras.metrics import categorical_crossentropy
from keras.utils import to_categorical
from sklearn.metrics import log_loss
from keras import backend as K

def onehot(x):
    to_categorical(x, 10)
    
def do_clip(arr, mx):
    return np.clip(arr, (1 - mx) / 9, mx)

In [118]:
valid_classes = valid_batches.classes
valid_labels = to_categorical(valid_classes, 10).astype('float32')

valid_preds_c = do_clip(valid_preds, 0.93)

print(valid_labels, 'valid_labels')
print(valid_labels.dtype, 'valid_labels type')
print(valid_labels.shape, 'valid_labels.shape')

print(valid_preds_c, 'valid_preds_c type')
print(valid_preds_c.dtype, 'valid_preds_c')
print(valid_preds_c.shape, 'valid_preds_c.shape')

cc = categorical_crossentropy(K.constant(valid_labels), K.constant(valid_preds_c))
valid_mean = np.mean(cc.eval(session=K.get_session()))

score = log_loss(valid_labels, valid_preds_c))

print(valid_mean, 'validation mean')

[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  1.]] valid_labels
float32 valid_labels type
(823, 10) valid_labels.shape
[[ 0.32545128  0.00920499  0.00777778 ...,  0.00777778  0.00777778
   0.61253059]
 [ 0.67435294  0.00777778  0.00777778 ...,  0.00777778  0.00777778
   0.28782073]
 [ 0.62463993  0.00777778  0.00777778 ...,  0.00777778  0.00777778
   0.28785765]
 ..., 
 [ 0.08136775  0.02805007  0.02607597 ...,  0.03632662  0.30010533
   0.50303668]
 [ 0.33036822  0.03377842  0.01220943 ...,  0.00777778  0.07727412
   0.47432873]
 [ 0.23242027  0.0451872   0.03887215 ...,  0.01095971  0.26165023
   0.29060861]] valid_preds_c type
float32 valid_preds_c
(823, 10) valid_preds_c.shape


ValueError: setting an array element with a sequence.

In [117]:
test_preds_c = do_clip(test_preds, 0.93)
test_preds_c

array([[ 0.04818407,  0.00777778,  0.01461318, ...,  0.00777778,
         0.0098451 ,  0.00777778],
       [ 0.89036   ,  0.00779618,  0.00777778, ...,  0.00777778,
         0.00777778,  0.06173703],
       [ 0.00777778,  0.0145776 ,  0.91266948, ...,  0.00777778,
         0.02415638,  0.00777778],
       ..., 
       [ 0.43349975,  0.3404859 ,  0.04322587, ...,  0.00777778,
         0.02240543,  0.0698687 ],
       [ 0.00777778,  0.01101675,  0.11367005, ...,  0.70606726,
         0.11160305,  0.00823002],
       [ 0.26121113,  0.05492519,  0.05338839, ...,  0.02645271,
         0.10260747,  0.22411396]], dtype=float32)

### Read model from cache and train

In [70]:
m1 = read_model(data_dir, 'architecture-2017-10-10-22.json', 'model-weights-2017-10-10-22.json')

m1.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
m1.optimizer.lr = 0.008
m1.fit_generator(t_batches, t_steps, epochs=1, validation_data=v_batches, validation_steps=v_steps)

Epoch 1/1
338/338 [==============================] - 183s - loss: 1.5169 - acc: 0.5037 - val_loss: 5.3738 - val_acc: 0.2224


In [50]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_31 (Batc (None, 224, 224, 3)       12        
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
batch_normalization_32 (Batc (None, 222, 222, 32)      128       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
batch_normalization_33 (Batc (None, 72, 72, 64)        256       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 24, 24, 64)        0         
__________